In [1]:
%%writefile matrix_mul.cu
#include <iostream>
#include <cstdlib>
#include <cuda_runtime.h>
using namespace std;

// CUDA kernel for matrix multiplication
__global__ void matMul(int* A, int* B, int* C, int N) {
    int row = threadIdx.y, col = threadIdx.x;
    int sum = 0;
    for (int i = 0; i < N; i++)
        sum += A[row * N + i] * B[i * N + col];
    C[row * N + col] = sum;
}

// Fill matrix with random values
void fill(int* m, int N) {
    for (int i = 0; i < N * N; i++) m[i] = rand() % 10;
}

// Print matrix
void print(const int* m, int N) {
    for (int i = 0; i < N * N; i++) {
        cout << m[i] << " ";
        if ((i + 1) % N == 0) cout << "\n";
    }
    cout << "\n";
}

int main() {
    const int N = 2, bytes = N * N * sizeof(int);
    int A[N*N], B[N*N], C[N*N], *dA, *dB, *dC;

    fill(A, N); fill(B, N);
    cout << "A:\n"; print(A, N);
    cout << "B:\n"; print(B, N);

    cudaMalloc(&dA, bytes); cudaMalloc(&dB, bytes); cudaMalloc(&dC, bytes);
    cudaMemcpy(dA, A, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(dB, B, bytes, cudaMemcpyHostToDevice);

    matMul<<<1, dim3(N, N)>>>(dA, dB, dC, N);
    cudaMemcpy(C, dC, bytes, cudaMemcpyDeviceToHost);

    cout << "A x B:\n"; print(C, N);

    cudaFree(dA); cudaFree(dB); cudaFree(dC);
    return 0;
}


Writing matrix_mul.cu


In [2]:
!nvcc -arch=sm_75 matrix_mul.cu -o matrix_mul
!./matrix_mul


A:
3 6 
7 5 

B:
3 5 
6 2 

A x B:
45 27 
51 45 

